## Inspired by https://github.com/amirmgh1375/iranian-license-plate-recognition

## Importing necessary libraries

In [1]:
import numpy as np
import cv2
# from google.colab.patches import cv2_imshow
import random
from PIL import Image
import os

## Numbers and Letters

In [2]:
# glyph_folder = "/content/glyph/b_roya"
glyph_folder = "./Glyphs/b_roya"

In [3]:
# Characters of Letters and Numbers in Plates
numbers = [str(i) for i in range(0, 10)]
# letters = ['ALEF', 'BE', 'PE', 'TE', 'SE', 'JIM', 'CHE', 'HE', 'KHE', 'DAL', 'ZAL', 'RE', 'ZE', 'ZHE', 'SIN','SHIN', 'SAD', 'ZAD', 'TA', 'ZA', 'EIN', 'GHEIN', 'FE', 'GHAF', 'KAF', 'GAF', 'LAM', 'MIM', 'NON', 'VAV', 'HA', 'YE']
letters = ["BE", "TE", "JIM", "DAL", "RE", "SIN", "SAD", "TA", "EIN", "GHAF", "LAM", "MIM", "NON", "VAV", "HE", "YE", "WHEEL"]


## Creating Glyphs (in case you already don't have them)

In [ ]:
threshold = 128 # pixels with value > thresh are considered 1 and 0 otherwise

source_path = "/content/SIN.png" # place character images in this folder, naming convention is SIN.png for letter and 1.png for numbres 

if not os.path.exists(save_folder):
  os.makedirs(save_folder)

In [ ]:
img = cv2.imread(source_path) # Read in the image and convert to grayscale
img_name = source_path.split("/")[-1].split('.')[0]

binary = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
binary = 255 * (binary > threshold).astype(np.uint8)
cv2_imshow(binary)
cv2.imwrite("/content/Glyphs/main/{}_trim.png".format(img_name), binary) # Save the image

## Generate Plates

In [4]:
# Returns an array containing a plate's letter and numbers:
# [number1, number2 , letter, number3, number4, number5]
def getNewPlate ():
    return [random.choice(numbers), 
            random.choice(numbers),
            random.choice(letters), 
            random.choice(numbers), 
            random.choice(numbers),
            random.choice(numbers),
            random.choice(numbers),
            random.choice(numbers)]
    # return plateFromName('37GAF853')


In [5]:
# Returns Address of a glyph image given font, and glyph name
def getGlyphAddress(glyphName):
    return os.path.join(glyph_folder, "{}.png".format(glyphName))

In [13]:
# Generate a plate as an array
# e.g. ['3', '7', 'GAF', '8', '5', '3']
plate = getNewPlate()
plate

['6', '6', 'HE', '0', '7', '8', '9', '4']

In [16]:
# Get Glyph images of plate characters
glyphImages = []
for glyph in plate:
    glyphImage = Image.open(getGlyphAddress(glyph)).convert("RGBA")
    # number.putalpha(255)
    glyphImages.append(glyphImage)

In [ ]:
# Create a blank image with size of templates 
# and add the background and glyph images
newPlate = Image.new('RGBA', (600,132), (0, 0, 0, 0))

if plate[2] in ["TE", "EIN"]:
  background = Image.open("/content/templates/template-ommomi.png").convert("RGBA")
else:
  background = Image.open("/content/templates/template-base.png").convert("RGBA")

newPlate.paste(background, (0,0))
# adding glyph images with 11 pixel margin
w = 0
for i, glyph in enumerate(glyphImages[:-2]):
    if i == 2:
        newPlate.paste(glyph, (70 + w,30), mask=glyph)
    else: newPlate.paste(glyph, (70 + w,25), mask=glyph)
    w += glyph.size[0] + 3

# last two digits
w = 0
for i, glyph in enumerate(glyphImages[-2:]):
  width, height = glyph.size[0], glyph.size[1]
  resized_glyph = glyph.resize((int(width * 0.75), int(height * 0.75)))
  newPlate.paste(resized_glyph, (485 + w, 50), mask=resized_glyph)
  w += glyph.size[0] - 10

_newPlate = newPlate.resize((312,70), Image.ANTIALIAS)
_newPlate.save("new_plate.png")